In [ ]:
# from kafka.admin import KafkaAdminClient, NewTopic


# admin_client = KafkaAdminClient(
#     bootstrap_servers="localhost:9092", 
#     client_id='tracking'
# )

# topic_list = [NewTopic(name="coordinates", num_partitions=3, replication_factor=3)]
# admin_client.create_topics(new_topics=topic_list, validate_only=False)

In [5]:
import json
import folium
import time
import re
import requests
from google.transit import gtfs_realtime_pb2
from json import dumps
from kafka import KafkaProducer
from kafka import KafkaConsumer

api_url = 'https://realtime.hsl.fi/realtime/vehicle-positions/v2/hsl'  # Replace with your API endpoint
KAFKA_SERVER = 'localhost:9092'
TOPIC = 'coordinates'


def fetch_coordinates():
    response = requests.get(api_url)
    if response.status_code == 200:
        # Fetch the GTFS-RT payload
        response = requests.get(api_url)
        payload = response.content

        # Create a GTFS-RT feed message
        feed = gtfs_realtime_pb2.FeedMessage()

        # Parse and decode the payload
        feed.ParseFromString(payload)

        # Create a list to hold the decoded entities
        decoded_entities = []

        # Iterate over the feed entities
        for entity in feed.entity:
            # Convert the entity to a dictionary
            if re.search(r'47/668', entity.id):
                entity_dict = {
                    'id': entity.id,
                    'vehicle': {
                        'trip': {
                            'route_id': entity.vehicle.trip.route_id,
                            'start_time': entity.vehicle.trip.start_time,
                            'start_date': entity.vehicle.trip.start_date
                            # Add other trip attributes as needed
                        },
                        'position': {
                            'latitude': entity.vehicle.position.latitude,
                            'longitude': entity.vehicle.position.longitude,
                            'bearing': entity.vehicle.position.bearing
                            # Add other trip attributes as needed
                        },
                        'position': {
                            'latitude': entity.vehicle.position.latitude,
                            'longitude': entity.vehicle.position.longitude,
                            'bearing': entity.vehicle.position.bearing,
                            'odometer': entity.vehicle.position.odometer,
                            'speed' : entity.vehicle.position.speed
                            # Add other trip attributes as needed
                        },
                        'stop_id': entity.vehicle.stop_id,
                        'current_status': entity.vehicle.current_status,
                        'timestamp': entity.vehicle.timestamp
                    }
                }


                # Append the entity dictionary to the list
                decoded_entities.append(entity_dict)

        # Convert the list of decoded entities to a JSON array
        json_array = json.dumps(entity_dict)
        # print(type(json_array))
        return json_array
    else:
        print(f"Error fetching coordinates: {response.status_code}")
        return None


def produce_kafka_messages():
    producer = KafkaProducer(bootstrap_servers=KAFKA_SERVER, 
                             api_version=(0,11,5),
                             value_serializer=lambda x: json.dumps(x).encode('utf-8'))

    while True:
        coordinates = fetch_coordinates()
        if coordinates:
            producer.send(TOPIC, value=coordinates)
            print(f"Message produced: {coordinates}")

        time.sleep(2)  # Wait for 5 seconds before fetching coordinates again

In [13]:
producer = KafkaProducer(bootstrap_servers=KAFKA_SERVER, 
                             api_version=(0,11),
                             value_serializer=lambda x: json.dumps(x).encode('utf-8'))

In [14]:
producer.send('coordinates', value="{'latitudew':123456}")

In [19]:
metrics = producer.metrics()

# Print the logs of all sent messages
for metric_name, metric_value in metrics.items():
    print(f"{metric_name}: {metric_value}")

kafka-metrics-count: {'count': 56.0}
producer-metrics: {'connection-close-rate': 0.0, 'connection-creation-rate': 0.0, 'select-rate': 9.197992727498292, 'io-wait-time-ns-avg': 108808932.59524418, 'io-wait-ratio': 1.0008237706979077, 'io-time-ns-avg': 1065.171359867794, 'io-ratio': 9.797438421603436e-06, 'connection-count': 1.0, 'bufferpool-wait-ratio': 0.0, 'batch-size-avg': 0.0, 'batch-size-max': -inf, 'compression-rate-avg': 0.0, 'record-queue-time-avg': 0.0, 'record-queue-time-max': -inf, 'produce-throttle-time-avg': 0.0, 'produce-throttle-time-max': -inf, 'record-send-rate': 0.0, 'records-per-request-avg': 0.0, 'byte-rate': 0.0, 'record-retry-rate': 0.0, 'record-error-rate': 0.0, 'record-size-max': -inf, 'record-size-avg': 0.0, 'requests-in-flight': 0.0, 'metadata-age': 90.11920849609375, 'network-io-rate': 0.0, 'outgoing-byte-rate': 0.0, 'request-rate': 0.0, 'request-size-avg': 0.0, 'request-size-max': -inf, 'incoming-byte-rate': 0.0, 'response-rate': 0.0, 'request-latency-avg': 0

In [38]:
produce_kafka_messages()

Message produced: {"id": "vehicle_position_47/668", "vehicle": {"trip": {"route_id": "1016", "start_time": "10:40:00", "start_date": "20240115"}, "position": {"latitude": 60.18600082397461, "longitude": 25.00366973876953, "bearing": 94.0, "odometer": 1533.0, "speed": 6.170000076293945}, "stop_id": "1420124", "current_status": 2, "timestamp": 1705308214}}
Message produced: {"id": "vehicle_position_47/668", "vehicle": {"trip": {"route_id": "1016", "start_time": "10:40:00", "start_date": "20240115"}, "position": {"latitude": 60.18619918823242, "longitude": 25.007049560546875, "bearing": 14.0, "odometer": 1730.0, "speed": 5.610000133514404}, "stop_id": "1420124", "current_status": 2, "timestamp": 1705308246}}
Message produced: {"id": "vehicle_position_47/668", "vehicle": {"trip": {"route_id": "1016", "start_time": "10:40:00", "start_date": "20240115"}, "position": {"latitude": 60.186500549316406, "longitude": 25.007068634033203, "bearing": 16.0, "odometer": 1765.0, "speed": 5.4000000953674

KeyboardInterrupt: 